<a href="https://colab.research.google.com/github/skyil7/SejongUniv_AI/blob/master/5.%EB%8B%A4%EC%A4%91%20%EB%B6%84%EB%A5%98%20(Multi-Class%20Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Class Classification
클래스가 3개 이상인 경우 Softmax 함수를 이용하여 분류

## Load Data

In [1]:
import torch
import torch.optim as optim

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

print(x_train[:5])
print(y_train[:5])

tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.]])
tensor([2, 2, 2, 1, 1])


In [2]:
print(y_train)
print(y_train.shape)
print(y_train.unsqueeze(1)) # unsqueeze 함수는 인수로 받은 위치에 새로운 차원을 삽입한다.
print(y_train.unsqueeze(1).shape)

nb_class = 3
nb_data = len(y_train)
y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)  # scatter(a,b,c): dimension a 에 대해 b에서 지정한 위치에 c 삽입
                                                # 언더바(_)를 함수뒤에 붙이면 in-place operation, 바꿔치기 연산이 되는데, 연산의 결과를 바로 y_one_hot에 저장하는 것이다.
                                                # (y_one_hot = y_one_hot.scatter.... 할 필요없이)

print(y_one_hot)

tensor([2, 2, 2, 1, 1, 1, 0, 0])
torch.Size([8])
tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])
torch.Size([8, 1])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


## Train Model

In [3]:
import torch.nn.functional as F # softmax가 포함된 클래스

# 모델 초기화
# feature 4개, 3개의 클래스
nb_class = 3
nb_data = len(y_train)

W = torch.zeros((4, nb_class), requires_grad=True) # 3개의 클래스에 대한 4개의 Feature별 가중치
b = torch.zeros(1, requires_grad=True) # 하나의 편향

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost 표현법 2번 예시
    # cross_entropy를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도 됨
    # cost = F.cross_entropy((x_train.matmul(W) + b), y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 1.036919
Epoch  200/1000 Cost: 1.010905
Epoch  300/1000 Cost: 0.991541
Epoch  400/1000 Cost: 0.974942
Epoch  500/1000 Cost: 0.959928
Epoch  600/1000 Cost: 0.946149
Epoch  700/1000 Cost: 0.933492
Epoch  800/1000 Cost: 0.921898
Epoch  900/1000 Cost: 0.911302
Epoch 1000/1000 Cost: 0.901625


## Evaluate Model

In [4]:
# 학습된 W,b를 통한 클래스 예측
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

print(hypothesis)
print(predict)
print(y_train)


# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([[0.3633, 0.1321, 0.5046],
        [0.1075, 0.3747, 0.5178],
        [0.0338, 0.4811, 0.4851],
        [0.0120, 0.5703, 0.4177],
        [0.9002, 0.0161, 0.0838],
        [0.1476, 0.7789, 0.0735],
        [0.8375, 0.0747, 0.0878],
        [0.8986, 0.0490, 0.0524]], grad_fn=<SoftmaxBackward>)
tensor([2, 2, 2, 1, 0, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([ True,  True,  True,  True, False,  True,  True,  True])
The model has an accuracy of 87.50% for the training set.
